<a href="https://colab.research.google.com/github/hoanglam372000/Applied-Data-Science-Group-7/blob/nguyenthiman/TXL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tiền xử lý bao gồm:

A. Làm thô
    Xóa \n 

    Xóa nhiều khoảng trắng

    xử lý html, url, tag, ký tự ẩn *** --> 

                 [ "<html_tag>",'<URL>','<HIDDEN>','<tag>',"<hash_tag>"]

    số  --> <NUMBER>

  Mở rộng don't --> do not

  Xử lý viết tắt omg --> // chưa 

  emoij --> từ 👍 --> :thumbs_up:

  kí tự đặc biệt --> <Special_Character>

  Viết thường

  sửa chính tả

  lemma
    
        caring --> care

  I'm not happy --> i am not not_happy (tính từ, trạng từ) // chưa

  tên --> <name>
       

Gán nhãn // chưa


## Import

In [ ]:
!pip install contractions
!pip install emoji
!pip install autocorrect

     |████████████████████████████████| 284 kB 8.0 MB/s 
     |████████████████████████████████| 321 kB 46.5 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85447 sha256=a81536db0d0fb8781f260e8c4184186e9dc1fc40dd9814d0e18c309a5cbc572a
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
     |████████████████████████████████| 170 kB 7.8 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=318fe913a8fcc28e6d458e091b0088973e07c73aba314fa954e8eb456d571380
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji
     |████████████████████████████████| 622 kB 7.6 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.5.0-py3-none-any.whl size=621851 sha256=37c503b7dc23a30432ea53eb37ea6f96722314b03253cde68ec999e59d73c305
  Stored in di

In [ ]:
import pandas as pd
import nltk
import string
import re
import contractions
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('names')
import emoji
from bs4 import BeautifulSoup
from autocorrect import Speller
names = nltk.corpus.names.words('male.txt')+nltk.corpus.names.words('female.txt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


# TXTXL


còn từ viết tắt



In [ ]:
tweet = " #gamegame36 didn't hi❤️👍My mom just texted me and told me that Rodney was *** chasing?? and fireflies... in (their) :)) backyard. Awwwww I'm miss him!!! @hoanglam 123 ://    http://twitpic.com/66pn1"
tweet2 = 'theira :)) <3'
def preprocessing_word(word):
  if word != emoji.demojize(word):
    return emoji.demojize(word)
  word = contractions.fix(word)
  temp=[ "<html_tag>",'<URL>','<HIDDEN>','<tag>',"<hash_tag>"]
  if word in temp:
    return word
  #số theo từ
  if word.isdigit():  
    return '<NUMBER>'
  try: 
    float(word)
    return '<NUMBER>'
  except:
    pass
  if word in names:
    return '<Name>'

  word=word.lower() # viết thườngthường
  spell_corrector = Speller(lang='en') #sửa chính tả theo từng từ 
  word=spell_corrector(word)

  # rút gọn theo từ
  lemmatizer = WordNetLemmatizer()
  wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
  pos_tagged = nltk.pos_tag(word.split())
  
  word = [lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged]
  word = ' '.join(word)

  word=word.translate(str.maketrans('', '', string.punctuation)) # chấm câu theo từ (vì tránh emoij)
  if word==''or word ==' ':
    return "<Special_Character>"
  if 'not' not in word and len(word.split())!=1:
    return word.split()[0]+'_'
  return word.replace(' ','_')

def preprocessing_text(tweet):
   raw_text = str(tweet).strip().replace('\n', ' ')#.lower()
   # emoij
   raw_text=emoji.demojize(raw_text)
   e_r=r':\w+_\w+:'
   emoij_list=re.findall(e_r,raw_text)
   for e in emoij_list:
     raw_text=raw_text.replace(e,' ' + emoji.emojize(e) + ' ') # "hi❤️👍"

   new_tweet = raw_text.split()
   len_=len(new_tweet)
   index2word = { w:new_tweet[w] for w in range(len_)}
   parser = BeautifulSoup(raw_text, "html.parser")
   raw_text = parser.get_text(separator = "<html_tag>")
   raw_text = re.sub(pattern=r'https?://\S+|www\.\S+', repl='<URL>', string=raw_text)
   raw_text = re.sub(pattern='\*+', repl='<HIDDEN>', string=raw_text)
   raw_text = re.sub(pattern=r'@\w+', repl='<tag>', string=raw_text)
   raw_text = re.sub(pattern=r'#\w+', repl='<hash_tag>', string=raw_text)
   word_list=raw_text.split()
   
   return index2word,dict(zip([*range(len_)],map(preprocessing_word,word_list)))
  #  return list(map(preprocessing_word,word_list))
   # (" ".join([word_list]))

index2word, p=preprocessing_text(tweet)
print(index2word)
print(p)
# preprocessing_word("texted")

{0: '#gamegame36', 1: "didn't", 2: 'hi', 3: '❤️', 4: '👍', 5: 'My', 6: 'mom', 7: 'just', 8: 'texted', 9: 'me', 10: 'and', 11: 'told', 12: 'me', 13: 'that', 14: 'Rodney', 15: 'was', 16: '***', 17: 'chasing??', 18: 'and', 19: 'fireflies...', 20: 'in', 21: '(their)', 22: ':))', 23: 'backyard.', 24: 'Awwwww', 25: "I'm", 26: 'miss', 27: 'him!!!', 28: '@hoanglam', 29: '123', 30: '://', 31: 'http://twitpic.com/66pn1'}
{0: '<hash_tag>', 1: 'do_not', 2: 'hi', 3: ':red_heart:', 4: ':thumbs_up:', 5: 'my', 6: 'mom', 7: 'just', 8: 'test', 9: 'me', 10: 'and', 11: 'told', 12: 'me', 13: 'that', 14: '<Name>', 15: 'be', 16: '<HIDDEN>', 17: 'chasing', 18: 'and', 19: 'fireflies', 20: 'in', 21: 'their', 22: '<Special_Character>', 23: 'backyard', 24: 'awwwww', 25: 'i_', 26: 'miss', 27: 'him', 28: '<tag>', 29: '<NUMBER>', 30: '<Special_Character>', 31: '<URL>'}
